- [Online IRT MCMC + Moment Matching](#MCMC-+-Moment-Matching)
- [Online IRT Search](#Search)

In [1]:
import probpy as pp
import numpy as np
import numba
import random

Utilities and data
---

In [2]:

@numba.jit(nopython=True, fastmath=True, forceobj=False)
def sigmoid(x):
    return (1 / (1 + np.exp(-x)))

def logit(x):
    return np.log(x / (1 - x))

student_skill = logit(0.7)

items = logit(np.array([0.4, 0.6, 0.8, 0.7]))  # difficulties


fast_p = pp.normal.fast_p
def likelihood(obs, item, skill): # IRT likelihood
    return fast_p(obs - sigmoid(skill - item), mu=0.0, sigma=0.6)
   
    
## IRT samples
samples = 100
obs, its = [], []
for i in range(samples):  
    item = items[np.random.randint(0, items.size)]
    outcome = float(np.random.rand() < sigmoid(student_skill - item))

    obs.append(outcome)
    its.append(item)

MCMC + Moment Matching
---

In [3]:
%%time


prior_skill = pp.normal.med(mu=0.0, sigma=5)

for i in range(samples)[:30]:
    prior_skill = pp.parameter_posterior((obs[i], its[i]), likelihood=likelihood, prior=prior_skill,
                                         mode="mcmc", match_moments_for=pp.normal,
                                         samples=20000, mixing=10000, batch=5, energy=0.1)
        
    
    print("observation", obs[i], "item", sigmoid(its[i]), "mode", sigmoid(pp.mode(prior_skill)))


observation 1.0 item 0.8 mode 0.7339889318635374
observation 1.0 item 0.6 mode 0.7719069600877001
observation 1.0 item 0.7 mode 0.8287788615879182
observation 0.0 item 0.8 mode 0.8008250578329901
observation 1.0 item 0.4 mode 0.8293684096605662
observation 0.0 item 0.6 mode 0.8779442183275237
observation 0.0 item 0.6 mode 0.8529397152841327
observation 1.0 item 0.6 mode 0.8708276483290379
observation 0.0 item 0.6 mode 0.8178713824384728
observation 0.0 item 0.8 mode 0.795990327053084
observation 1.0 item 0.8 mode 0.8388065855213576
observation 1.0 item 0.8 mode 0.8708066308122976
observation 0.0 item 0.6 mode 0.8898582949067646
observation 0.0 item 0.8 mode 0.8510457151562363
observation 1.0 item 0.4 mode 0.8505800233244247
observation 1.0 item 0.6 mode 0.8759193667596842
observation 0.0 item 0.4 mode 0.8419322694063648
observation 0.0 item 0.8 mode 0.8284515968468291
observation 1.0 item 0.8 mode 0.840052085252072
observation 1.0 item 0.4 mode 0.8530815197887954
observation 1.0 item 0

Search
---

In [7]:
%%time


prior_skill = pp.normal.med(mu=0.0, sigma=10)

for i in range(samples)[:30]:
    prior_skill = pp.parameter_posterior((obs[i], its[i]), 
                                         likelihood=likelihood, prior=prior_skill,
                                         mode="search",
                                         samples=300, batch=5,
                                         volume=100, energy=0.1,
                                         variance=2.0)
    
    mode = sigmoid(pp.mode(prior_skill)[0])
    print("observation", obs[i], "item", sigmoid(its[i]), "mode", mode)

observation 1.0 item 0.8 mode [0.84559671]
observation 1.0 item 0.6 mode [0.85488598]
observation 1.0 item 0.7 mode [0.86590529]
observation 0.0 item 0.8 mode [0.83819878]
observation 1.0 item 0.4 mode [0.83750717]
observation 0.0 item 0.6 mode [0.80366394]
observation 0.0 item 0.6 mode [0.76456734]
observation 1.0 item 0.6 mode [0.78273926]
observation 0.0 item 0.6 mode [0.73684243]
observation 0.0 item 0.8 mode [0.7093001]
observation 1.0 item 0.8 mode [0.75373758]
observation 1.0 item 0.8 mode [0.79069475]
observation 0.0 item 0.6 mode [0.74213742]
observation 0.0 item 0.8 mode [0.71955778]
observation 1.0 item 0.4 mode [0.72735861]
observation 1.0 item 0.6 mode [0.7503357]
observation 0.0 item 0.4 mode [0.70214005]
observation 0.0 item 0.8 mode [0.67802595]
observation 1.0 item 0.8 mode [0.72682425]
observation 1.0 item 0.4 mode [0.7377039]
observation 1.0 item 0.4 mode [0.75198597]
observation 1.0 item 0.4 mode [0.75640111]
observation 1.0 item 0.6 mode [0.76595459]
observation 1.